<a href="https://colab.research.google.com/github/babakbadakhshan/urban-network-analysis/blob/main/Mashhad_Travel_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import osmnx as ox
import pandana
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [82]:
# Select city and crs
cityname = 'مشهد, Iran'
crs = 32640

In [83]:
# Get graph by geocoding
graph = ox.graph_from_place(cityname, network_type="walk")

# Project graph
graph = ox.projection.project_graph(graph, to_crs=crs)

In [84]:
# Select points of interest based on osm tags
tags = {
    'amenity': ['cafe', 'bar', 'pub', 'restaurant'],
    'shop': ['bakery', 'convenience', 'supermarket', 'mall', 'department_store', 'clothes', 'fashion', 'shoes'],
    'leisure': ['fitness_centre']
}

# Get amentities from place
pois = ox.geometries.geometries_from_place(cityname, tags=tags)

# Project pois
pois = pois.to_crs(epsg=crs)

In [138]:
# Walking time
walk_time = 110
# Walking speed
walk_speed = 4.5

# Set a uniform walking speed on every edge
for u, v, data in graph.edges(data=True):
    data['speed_kph'] = walk_speed
graph = ox.add_edge_travel_times(graph)

# Extract node/edge GeoDataFrames, retaining only necessary columns (for pandana)
nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]


In [139]:
# Construct the pandana network model
network = pandana.Network(
    node_x=nodes['x'],
    node_y=nodes['y'],
    edge_from=edges['u'],
    edge_to=edges['v'],
    edge_weights=edges[['travel_time']]
)

# Extract centroids from the pois' geometries
centroids = pois.centroid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [140]:
# Specify a max travel distance for analysis
# Minutes -> seconds
maxdist = walk_time * 60

# Set the pois' locations on the network
network.set_pois(
    category='pois',
    maxdist=maxdist,
    maxitems=10,
    x_col=centroids.x,
    y_col=centroids.y
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [141]:
# calculate travel time to 10 nearest pois from each node in network
distances = network.nearest_pois(
    distance=maxdist,
    category='pois',
    num_pois=10
)

distances.astype(int).head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,1,2,3,4,5,6,7,8,9,10
osmid,,,,,,,,,,
259990630,2381,4577,4587,4684,4820,4848,5144,5144,5167,5180
708236647,2292,4489,4498,4596,4731,4760,5055,5055,5078,5091
260096366,169,171,196,205,269,291,323,335,366,457
260096367,120,154,220,240,246,272,320,385,417,508
266139962,113,213,227,252,261,278,347,379,410,422


In [151]:
# Calculate the average distance to the 10 nearest POIs for each node
average_distances = distances.mean(axis=1)

# Add the average distances to the nodes GeoDataFrame
nodes['avg_distance_to_POIs'] = average_distances

# Save nodes with average distance as a gpkg
nodes_gdf = gpd.GeoDataFrame(nodes, geometry=gpd.points_from_xy(nodes['x'], nodes['y']), crs=f"EPSG:{crs}")
nodes_gdf.to_file('15-mashhad.gpkg')